In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import csv
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA,IncrementalPCA, FactorAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import validation_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve

plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing


from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.python.keras import regularizers
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

**I have processed the data and saved in csv for quick use**

In [3]:
apptrain_encoded=pd.read_csv("apptrain_encoded.csv")
apptest_encoded=pd.read_csv("apptest_encoded.csv")
apptrain_encoded.drop("Unnamed: 0",axis=1,inplace=True)
apptest_encoded.drop("Unnamed: 0",axis=1,inplace=True)

In [3]:
apptrain_encoded.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Missing,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,1,0,0,1,0,1
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0,0,1,0,0,0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0,0,1,0,0,0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
apptest_encoded.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Missing,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Missing,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0,0,1,0,0,1,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,1,0,0,0,0,0,0,1,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,1,0,0,0,0,0,0,1,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0,1,0,0,0,1,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,1,0,0,0,0,0,0,1,0,0


In [4]:
kerasSeqModel1=Sequential()

In [9]:
kerasSeqModel1.add(Dense(32, activation='relu',kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), input_shape =(248,)))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(32, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.4))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.2))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.5))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.3))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8, kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dropout(0.1))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(8,kernel_initializer='VarianceScaling', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation='relu'))
kerasSeqModel1.add(Dense(8, activation='relu'))
kerasSeqModel1.add(Dense(1, activation='sigmoid'))

In [10]:
kerasSeqModel1.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['categorical_accuracy'])

# case 1: accuracy is 100% but model doesnt learn

In [12]:
kerasSeqModel1.fit(apptrain_encoded.drop("TARGET", axis=1), apptrain_encoded["TARGET"],
          epochs=10,
          validation_split = 0.2)

Train on 246008 samples, validate on 61503 samples
Epoch 1/10
246008/246008 [==============================] - 59s 241us/step - loss: 1.6483 - categorical_accuracy: 1.0000 - val_loss: 0.4351 - val_categorical_accuracy: 1.0000
Epoch 2/10
246008/246008 [==============================] - 64s 260us/step - loss: 0.3100 - categorical_accuracy: 1.0000 - val_loss: 0.3293 - val_categorical_accuracy: 1.0000
Epoch 3/10
246008/246008 [==============================] - 62s 252us/step - loss: 0.2991 - categorical_accuracy: 1.0000 - val_loss: 0.2789 - val_categorical_accuracy: 1.0000
Epoch 4/10
246008/246008 [==============================] - 62s 253us/step - loss: 0.2823 - categorical_accuracy: 1.0000 - val_loss: 0.2781 - val_categorical_accuracy: 1.0000
Epoch 5/10
246008/246008 [==============================] - 62s 254us/step - loss: 0.2819 - categorical_accuracy: 1.0000 - val_loss: 0.2777 - val_categorical_accuracy: 1.0000
Epoch 6/10
246008/246008 [==============================] - 63s 255us/step

In [13]:
kerasprobs=kerasSeqModel1.predict(apptest_encoded)

# same probability for all cases = mission failed

In [14]:
kerasprobs

array([[0.08080583],
       [0.08080583],
       [0.08080583],
       ...,
       [0.08080583],
       [0.08080583],
       [0.08080583]], dtype=float32)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(apptrain_encoded.drop("TARGET", axis=1), apptrain_encoded["TARGET"], test_size=0.33, random_state=42)

# case 2: added conv2D, What is the issue here?

In [16]:
kerasSeqModel1.add(Conv2D(16, activation='relu', kernel_size=(3,3), input_shape=(248,)))
kerasSeqModel1.add(Dense(8, activation='relu',input_shape =(248,)))
kerasSeqModel1.add(Dense(8, activation='relu'))
#kerasSeqModel1.add(Dropout(0.5))
kerasSeqModel1.add(Dense(1, activation='sigmoid'))

ValueError: Input 0 of layer conv2d_1 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1]

**same issue, with some minor parameter tweaking**

In [17]:
kerasSeqModel1.add(Conv2D(32, activation='relu', kernel_size=(1,1), input_shape=(248,)))
kerasSeqModel1.add(Dense(8, activation='relu',input_shape =(248,)))
kerasSeqModel1.add(Dense(18, activation='relu'))
#kerasSeqModel1.add(Dropout(0.5))
kerasSeqModel1.add(Dense(1, activation='sigmoid'))

ValueError: Input 0 of layer conv2d_2 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1]